### Nonlinear paper 

[Link to paper](./references/Magnon-mediated%20optical%20frequency%20comb%20in%20a%20cavity%20optomagnonical%20system.pdf)

In [1]:
using QuantumCumulants
using OrdinaryDiffEq, ModelingToolkit
using DifferentialEquations
using PyPlot
import LinearAlgebra as la

In [9]:
@cnumbers ωm Δa Δm εm εc εp κ γ g
@syms t::Real
@register_symbolic ξ(t)

# Define the Fock spaces
hmw = FockSpace(:cavity); hm = FockSpace(:cavity)
h = hmw⊗hm

ωw = ωm - Δm
ω0 = 2*pi*193e12
ωc = ω0 - Δa
ωp = 2*pi*191e12
Ω = ωp - ωc

# Define the fundamental operators
@qnumbers a::Destroy(h,1) m::Destroy(h,2)
#            resonator          magnon        magnon YIG

ħ=1

# Define the Hamiltonian
Ham = ℏ*Δa*(a'*a) + ℏ*Δm*(m'*m) + im*ℏ*γ^.5*εm*((m'-m)) +
      ℏ*g*a'*a*((m'*exp(im*ωw*t)) + (m*exp(-im*ωw*t))) +
      im*ℏ*κ^.5*((εc + εp*ξ(t)*exp(-im*Ω*t))*(a') - (εc + εp*ξ(t)*exp(im*Ω*t))*(a))

# Collapse operators
J = [a, m]#, b3]
rates = [κ, γ]#, 2γ3]


# Derive a set of equations
ops = [a, m]#, b3]
eqs = meanfield(ops, Ham, J; rates=rates, order=1)

∂ₜ(⟨a⟩) = (0 - 1im)*⟨a⟩*Δa - 0.5⟨a⟩*κ + (εc + exp((0 - 1im)*t*(-1.256637061435925e13 + Δa))*Main.ξ(t)*εp)*(κ^0.5) + (0 - 1im)*g*exp((0 - 1im)*t*(-Δm + ωm))*⟨m⟩*⟨a⟩ + (0 - 1im)*g*⟨m′⟩*exp((0 + 1im)*t*(-Δm + ωm))*⟨a⟩
∂ₜ(⟨m⟩) = (γ^0.5)*εm + (0 - 1im)*⟨m⟩*Δm - 0.5⟨m⟩*γ + (0 - 1im)*g*⟨a′⟩*exp((0 + 1im)*t*(-Δm + ωm))*⟨a⟩


In [21]:
t0 = 1e-5
tp = 1e-9

function ξ(t)
    return exp(2*log(2)*((t-t0)/tp)^2)
end

# Numerical solution
ps = (ωm,Δa,Δm,εm,εc,εp,κ,γ,g)
@named sys = ODESystem(eqs)
u0 = zeros(ComplexF64, length(eqs))
p0 = ps .=> (2*pi*5e9,0.0,0.0,1e6,1e6,1e6,2*pi*60*1e6,2*pi*6e6,2*pi*10.4)

prob = ODEProblem(sys,u0,(0.5996,0.6004),p0)

ODEProblem with uType Vector{ComplexF64} and tType Float64. In-place: true
timespan: (0.5996, 0.6004)
u0: 2-element Vector{ComplexF64}:
 0.0 + 0.0im
 0.0 + 0.0im

In [22]:
sol = solve(prob,RK4())

┌ Warning: Automatic dt set the starting dt as NaN, causing instability. Exiting.
└ @ OrdinaryDiffEqCore C:\Users\freak\.julia\packages\OrdinaryDiffEqCore\3Talm\src\solve.jl:649
┌ Warning: NaN dt detected. Likely a NaN value in the state, parameters, or derivative value caused this outcome.
└ @ SciMLBase C:\Users\freak\.julia\packages\SciMLBase\XzPx0\src\integrator_interface.jl:583


retcode: DtNaN
Interpolation: 3rd order Hermite
t: 1-element Vector{Float64}:
 0.5996
u: 1-element Vector{Vector{ComplexF64}}:
 [0.0 + 0.0im, 0.0 + 0.0im]